In [0]:
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

In [4]:
view_line_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [5]:
import problem_unittests as tests
from collections import Counter
def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """

    counts=Counter(text)
    vocab=sorted(counts,key=counts.get,reverse=True)
    vocab_to_int={word: ii for ii,word in enumerate(vocab)}

    int_to_vocab={ii:word for word,ii in vocab_to_int.items()}
    # return tuple
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    token_dict={
        '.':'||PERIOD||',
        ',':'||COMMA||',
        '"':'||QUOTATION||',
        ';':'||SEMICOLON||',
        '!':'||EXCLAMATION||',
        '?':'||QUESTION||',
        '(':'||LEFTPARENTHESIS||',
        ')':'||RIGHTPARENTHESIS||',
        '-':'||DASH||',
        '\n':'||RETURN||'
    }
    return token_dict

tests.test_tokenize(token_lookup)

Tests Passed


In [0]:
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [0]:
# This cell acts as a checkpoint
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [0]:
# check if gpu is available
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

In [0]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    n_batches=len(words)//batch_size
    words=words[:n_batches*batch_size]
    y_len=len(words)-sequence_length
    x,y=[],[]
    for idx in range(0,y_len):
      idx_end=sequence_length+idx
      x_batch=words[idx:idx_end]
      batch_y=words[idx_end]
      x.append(x_batch)
      y.append(batch_y)
    # return a dataloader

    data=TensorDataset(torch.from_numpy(np.asarray(x)),torch.from_numpy(np.asarray(y)))
    dataloader=DataLoader(data,shuffle=False,batch_size=batch_size)
    return dataloader



In [11]:
# test dataloader
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 0,  1,  2,  3,  4],
        [ 1,  2,  3,  4,  5],
        [ 2,  3,  4,  5,  6],
        [ 3,  4,  5,  6,  7],
        [ 4,  5,  6,  7,  8],
        [ 5,  6,  7,  8,  9],
        [ 6,  7,  8,  9, 10],
        [ 7,  8,  9, 10, 11],
        [ 8,  9, 10, 11, 12],
        [ 9, 10, 11, 12, 13]])

torch.Size([10])
tensor([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14])


In [12]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        
        # set class variables
        self.vocab_size=vocab_size
        self.output_size=output_size
        self.embedding_dim=embedding_dim
        self.hidden_dim=hidden_dim
        self.n_layers=n_layers
        self.dropout=dropout

        # define model layers
        self.em1=nn.Embedding(self.vocab_size,self.embedding_dim)
        self.lstm=nn.LSTM(self.embedding_dim,self.hidden_dim,self.n_layers,dropout=self.dropout,batch_first=True)
        self.fc=nn.Linear(self.hidden_dim,self.output_size)

    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size=nn_input.size(0)

        nn_input=nn_input.long()
        embeds=self.em1(nn_input)
        lstm_out,hidden=self.lstm(embeds,hidden)
        lstm_out=lstm_out.contiguous().view(-1,self.hidden_dim)
        out=self.fc(lstm_out)
        out=out.view(batch_size,-1,self.output_size)
        out=out[:,-1]
        # return one batch of output word scores and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight=next(self.parameters()).data
        if train_on_gpu:
          hidden=(weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda())
        else:
          hidden=(weight.new(self.n_layers,batch_size,self.hidden_dim).zero_(),
                  weight.new(self.n_layers,batch_size,self.hidden_dim).zero_())
        return hidden

tests.test_rnn(RNN, train_on_gpu)

Tests Passed


In [13]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param rnn: The PyTorch Module that holds the neural network
    :param optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    
    # move data to GPU, if available
    if train_on_gpu:
      inp,target=inp.cuda(),target.cuda()
    # perform backpropagation and optimization
    hidden=tuple([each.data for each in hidden])
    rnn.zero_grad()
    output,hidden=rnn(inp,hidden)
    loss=criterion(output,target)
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(),5)
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden


tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


In [0]:


def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of unique tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.


In [0]:
# Data params
# Sequence Length
sequence_length =  15 # of words in a sequence
# Batch Size
batch_size =  128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [0]:
# Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(vocab_to_int)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 250
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 2000

In [25]:

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 4.929815348267555

Epoch:    1/15    Loss: 4.488612960934639

Epoch:    1/15    Loss: 4.352982956767082

Epoch:    2/15    Loss: 4.111203026578754

Epoch:    2/15    Loss: 3.9347216089963912

Epoch:    2/15    Loss: 3.893580560684204

Epoch:    3/15    Loss: 3.8089283254108017

Epoch:    3/15    Loss: 3.7286862382888795

Epoch:    3/15    Loss: 3.6930458450317385

Epoch:    4/15    Loss: 3.6390796085252273

Epoch:    4/15    Loss: 3.585211606144905

Epoch:    4/15    Loss: 3.5529409527778624

Epoch:    5/15    Loss: 3.5249800106907148

Epoch:    5/15    Loss: 3.4664311213493346

Epoch:    5/15    Loss: 3.456453880548477

Epoch:    6/15    Loss: 3.4326326990866596

Epoch:    6/15    Loss: 3.3784717894792555

Epoch:    6/15    Loss: 3.3766451362371446

Epoch:    7/15    Loss: 3.3631483782333826

Epoch:    7/15    Loss: 3.31347090113163

Epoch:    7/15    Loss: 3.3108875020742414

Epoch:    8/15    Loss: 3.304568327340797

Epoch:    8/15

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
# Another Checkpoint
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

In [0]:
import torch.nn.functional as F
import numpy as np
def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        if(train_on_gpu):
            current_seq = current_seq.cpu() # move to cpu
        # the generated word becomes the next "current sequence" and the cycle can continue
        if train_on_gpu:
            current_seq = current_seq.cpu()
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

In [30]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'kramer' # name for starting the script

pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

kramer:.

jerry: what is the bubble boy?

george: yes.

george: i know, i don't want to talk.

elaine: oh, yeah, i am going to get out.

george: i don't want to see that.

jerry: oh, yeah!

jerry: so what are you gonna do?

elaine: no. it's just a little.

jerry: no, no.

george: what?

george: i don't know. it's not complicated.

kramer: yeah.

elaine: you know, i think i have to go out with you anymore.

george: what do you have to do with that?

jerry: no.

jerry: i know.

kramer: you know, the library is going to happen to trial, you know,"

jerry: what?

jerry: i can't get the arrest.

elaine: well, i was trying to smother you, but it was a little more.

elaine: what?

kramer: well...

jerry: i know how to do it was wet.

jerry: oh.

kramer: hey, you can go to the beach to eat in the mood.

frank: jackass, jackie.

elaine: so what do you think?

george: yeah.

elaine: oh, yeah.

kramer: well, i'm not gonna be honest with this.

jerry: what is the matter?

jerry: no.

jerry: what a

In [0]:
# save script to a text file
f =  open("generated_script_2.txt","w")
f.write(generated_script)
f.close()